In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1042,2024-06-11,Lituânia Lkl,12:50,Lietkabelis,BC Wolves,1.55,2.39,169.5,1.87,1.79,0.0,0.00,0.00,GC4cYtEc,0.645161,0.418410,0.534759,0.558659,0.063571,317.806,162.680041,0.511885,0.6,1.341641,2.236068,113.15,4.012,2.155834,0.537346,-37.0,224.364,59.460512,0.265018,1.2,1.643168,1.369306,193.59,2.628,0.633577,0.241087,3.0,73,81,1.55,2.39,157.196,221.946,0.301507,0.030912,NaN,-3.81,-0.762,-0.721785,0.665355,0.4,-0.265355,0.48,0.096,14.479167,0.540863,0.5,-0.040863
1043,2024-06-11,Venezuela Superliga,21:00,Broncos,Trotamundos,1.83,1.89,156.5,1.85,1.85,-1.5,1.94,2.07,AuuJ9MzG,0.546448,0.529101,0.540541,0.540541,0.075549,141.236,50.779969,0.359540,3.0,0.000000,0.000000,131.60,1.634,0.554374,0.339274,36.0,132.680,64.133322,0.483368,1.8,1.643168,0.912871,107.73,1.550,0.584252,0.376937,19.0,94,63,1.40,1.71,0.000,113.502,0.022810,0.000000,0.045847,0.00,0.000,inf,0.000000,0.0,0.000000,-2.15,-0.430,-2.069767,0.000000,0.0,0.000000
1044,2024-06-11,Venezuela Superliga,20:00,Toros de Aragua,Spartans,2.62,1.45,163.5,1.84,1.84,0.0,0.00,0.00,xjPc2vTk,0.381679,0.689655,0.543478,0.543478,0.071335,272.972,90.885766,0.332949,0.0,0.000000,NaN,243.60,3.364,1.435994,0.426871,-50.0,181.824,60.608782,0.333338,0.6,1.341641,2.236068,142.56,2.262,0.586703,0.259373,-12.0,87,72,2.80,1.98,261.572,159.940,0.406543,0.000000,NaN,-1.75,-0.350,-4.628571,0.000000,0.0,0.000000,-5.00,-1.000,-0.450000,0.000000,0.0,0.000000
1045,2024-06-11,Venezuela Superliga,20:00,Guaiqueries,Marinos,2.50,1.49,155.5,1.85,1.85,0.0,0.00,0.00,pjXk68oL,0.400000,0.671141,0.540541,0.540541,0.071141,160.128,44.968902,0.280831,1.2,1.643168,1.369306,151.30,2.064,0.747951,0.362379,16.0,111.950,17.288490,0.154430,2.4,1.341641,0.559017,96.94,1.434,0.215592,0.150343,25.0,85,74,1.78,1.31,138.736,115.090,0.357984,0.000000,NaN,-0.78,-0.156,-9.615385,0.000000,0.0,0.000000,-0.03,-0.006,-81.666667,0.000000,0.0,0.000000
1046,2024-06-11,Venezuela Superliga,20:00,Gaiteros del Zulia,Diablos,1.44,2.65,150.5,1.86,1.80,0.0,0.00,0.00,xQTs8UG8,0.694444,0.377358,0.537634,0.555556,0.071803,186.074,63.504285,0.341285,0.6,1.341641,2.236068,201.60,2.614,1.195734,0.457435,-22.0,244.370,49.636631,0.203121,1.2,1.643168,1.369306,249.10,3.018,0.850218,0.281716,-24.0,63,94,3.20,2.65,0.000,223.516,0.418386,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-1.650000,0.000000,0.0,0.000000
1047,2024-06-11,Venezuela Superliga,20:00,Centauros,Heroes de Falcon,1.81,1.91,147.5,1.85,1.85,-1.5,1.92,2.10,A3N50Ic2,0.552486,0.523560,0.540541,0.540541,0.076046,137.078,41.477040,0.302580,0.0,0.000000,NaN,126.08,1.960,0.679080,0.346470,-44.0,141.502,32.440193,0.229256,1.8,1.643168,0.912871,174.60,1.680,0.365445,0.217527,14.0,64,90,1.97,1.94,157.412,0.000,0.038016,0.000000,0.063323,-1.92,-0.384,-2.109375,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1048,2024-06-11,Venezuela Superliga,19:30,Frontinos,Pioneros,2.22,1.61,153.5,1.87,1.80,0.0,0.00,0.00,I5rR705S,0.450450,0.621118,0.534759,0.555556,0.071568,295.586,267.979924,0.906606,1.2,1.643168,1.369306,82.25,3.948,3.270401,0.828369,-12.0,142.774,32.020335,0.224273,1.8,1.643168,0.912871,139.04,1.958,0.546370,0.279045,-31.0,47,79,1.75,1.76,306.936,0.000,0.225240,0.026974,NaN,9.12,1.824,0.668860,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1049,2024

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1042,12:50,Lituânia Lkl,Lietkabelis,BC Wolves,1.55,Back Home
1047,20:00,Venezuela Superliga,Centauros,Heroes de Falcon,1.81,Back Home
